In [2]:
import torch

In [6]:
state = torch.randn(size=(16, 2, 15, 15))
conv = torch.nn.Conv2d(in_channels=2, out_channels=32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
feature = conv(state)
feature.shape

torch.Size([16, 32, 15, 15])

In [10]:
corner_list = [state[:, :, 0:5, 0:5], state[:, :, 10:15, 0:5], state[:, :, 10:15, 10:15], state[:, :, 0:5, 10:15]]
edge_list = [state[:, :, 0:5, 5:10], state[:, :, 5:10, 0:5], state[:, :, 5:10, 10:15], state[:, :, 10:15, 5:10]]
center_list = [state[:, :, 5:10, 5:10]]
corner_list, edge_list, center_list

([tensor([[[[ 0.0311, -2.0320,  0.5493,  0.5573, -0.5942],
            [ 0.3782,  0.9593,  0.9254, -1.2914, -1.3930],
            [-0.3509, -1.2918,  0.0526, -1.0011, -0.0410],
            [ 0.3010,  0.5699, -0.8985, -2.3047, -0.7472],
            [ 0.1057,  0.2541,  0.6816,  0.2712,  0.2204]],
  
           [[-0.9727, -0.1993, -0.0412, -0.0051,  3.6895],
            [ 1.2347, -1.9481, -0.2183,  0.6505,  0.6294],
            [-0.1582,  1.4069, -0.2041, -0.8326, -0.1862],
            [ 0.3057,  0.7492, -0.6493, -0.7721, -0.0095],
            [-0.6775, -0.5174, -1.4356, -0.0908,  0.3961]]],
  
  
          [[[ 0.0771, -0.7884, -1.6420, -0.1964,  1.7549],
            [ 2.0023,  0.3729, -0.9442,  0.6558,  2.9297],
            [ 0.0788, -0.0737,  0.6979, -1.5142,  1.9765],
            [-0.8575,  1.3155,  0.5290,  0.3139, -1.3751],
            [-2.8822, -1.5941, -0.5592, -0.1666,  1.8442]],
  
           [[ 1.4724,  0.5921,  0.2237, -0.9841, -0.3336],
            [-1.1460,  0.2273,  1.8249, 

In [ ]:
# means
means = torch.mean(torch.flatten(feature, start_dim=2), dim=2, keepdim=False)
gru = torch.nn.Linear(in_features=16, out_features=3)
gate = torch.softmax(gru(means), dim=1)

corner_expert = torch.nn.Linear(in_features=16, out_features=16, bias=True)
edge_expert = torch.nn.Linear(in_features=16, out_features=16, bias=True)
center_expert = torch.nn.Linear(in_features=16, out_features=16, bias=True)

out = torch.zeros(size=(16, 3, 32))

for c in corner_list:
    out[:, 0, :] += corner_expert(c)
for e in edge_list:
    out[:, 1, :] += edge_expert(e)
for c in center_list:
    out[:, 2, :] += center_expert(c)

out = out * gate.unsqueeze(1)

final = torch.nn.Linear(in_features=16, out_features=1, bias=True)

out = final(out)